In [40]:
#Поиск видео по конкретному запросу
q = "эко привычки"
type = 'video'
#для поиска видео с учетом даты
publishedAfter ='2000-01-01T00:00:00Z'
publishedBefore = '2021-04-01T00:00:00Z'

In [1]:
#импорт библиотек
import googleapiclient.discovery as api
import pandas
import re

In [2]:
API_KEY = "KEY"
api_service_name = "youtube"
api_version = "v3"
youtube = api.build(
    api_service_name, api_version, developerKey = API_KEY)

In [17]:
#Значения аргумента order
order_list = ['date', 'rating', 'relevance', 'title', 'videoCount','viewCount']
order_list

['date', 'rating', 'relevance', 'title', 'videoCount', 'viewCount']

In [27]:
#Первый заход без аргумента order
request = youtube.search().list(
    part="snippet",
    maxResults=50,
    q=q,
    type=type
)
response = request.execute()
df_supplemented = pandas.json_normalize(response['items'])
#Цикл для прохода по всем следующим страницам с выдачей
i=0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
      part="snippet",
      maxResults=50,
      q=q,
      type=type,
      pageToken = response['nextPageToken']
    )
    response = request.execute()
    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i+=1
    df_additional = pandas.json_normalize(response['items'])
    df_supplemented = pandas.concat([df_supplemented, df_additional])
print(f"Искомых объектов {response['pageInfo']['totalResults']}, \
       а найденных БЕЗ включения каких-либо значений аргемнта order {len(df_supplemented.drop_duplicates('id.videoId'))}")
#Найдено 519 видео,поэтому используем поиск видео с пересортировкой

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Итерация БЕЗ включения аргумента order №10
Искомых объектов 56257,        а найденных БЕЗ включения каких-либо значений аргемнта order 519


In [36]:
#Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи
i = 0
for order in order_list:
    #Для остановки алгоритма, если все исходныеобъекты найдены
    #БЕЗ включения каких-либо значений аргумента order
    if len(df_supplemented.drop_duplicates('id.videoId')) < response['pageInfo']['totalResults']:
        #Первый заход с каждым значением аргумента order
        request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q=q,
        type=type,
        order=order
        )
        response = request.execute()
        #Для визуализации процесса
        print (f'Итерация №{i}, "order"{order}, "items"{len(response["items"])}')
        i+=1
        #Занесение собранных данных в итоговую таблицу df_supplemented
        df = pandas.json_normalize(response['items'])
        df_supplemented = pandas.concat([df, df_supplemented])
        #Заходы с тем же значением агрумента order на следующие страницы
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates('id.videoId')) < response['pageInfo']['totalResults'])\
        & (len(response["items"])>0):
            #второе условие для остановки алгоритма, если все искомые объекты найдены
            #БЕЗ какой-то из следующих страниц
            #третье условие для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов
            request = youtube.search().list(
            part="snippet",
            maxResults=50,
            q=q,
            type=type,
            pageToken = response['nextPageToken'],
            order=order
            )
            response = request.execute()
            #Для визуализации процесса
            print (f'Итерация №{i}, "order"{order}, "items"{len(response["items"])}')
            i+=1
            #Занесение собранных данных в итоговую таблицу df_supplemented
            df = pandas.json_normalize(response['items'])
            df_supplemented = pandas.concat([ df_supplemented, df_additional])
        #Сохранение составленных для каждого значения аргумента order таблиц в Excel
        df_supplemented.to_excel(f'Video_sorted_by{order}.xlsx')
    else:
        print('Все искомые объекты найдены без включения некоторых значений аргумента order(в т.ч. вообще БЕЗ них)')

Итерация №0, "order"date, "items"50
Итерация №1, "order"date, "items"50
Итерация №2, "order"date, "items"50
Итерация №3, "order"date, "items"50
Итерация №4, "order"date, "items"50
Итерация №5, "order"date, "items"0
Итерация №6, "order"rating, "items"50
Итерация №7, "order"rating, "items"50
Итерация №8, "order"rating, "items"50
Итерация №9, "order"rating, "items"50
Итерация №10, "order"rating, "items"49
Итерация №11, "order"rating, "items"0
Итерация №12, "order"relevance, "items"50
Итерация №13, "order"relevance, "items"50
Итерация №14, "order"relevance, "items"50
Итерация №15, "order"relevance, "items"50
Итерация №16, "order"relevance, "items"50
Итерация №17, "order"relevance, "items"50
Итерация №18, "order"relevance, "items"50
Итерация №19, "order"relevance, "items"50
Итерация №20, "order"relevance, "items"50
Итерация №21, "order"relevance, "items"50
Итерация №22, "order"relevance, "items"25
Итерация №23, "order"relevance, "items"0
Итерация №24, "order"title, "items"50
Итерация №25, "

In [39]:
#Удалим дубликаты видео по их ID и запишем в Excel
df_supplemented = df_supplemented.drop_duplicates('id.videoId')
df_supplemented.index = range(1, len(df_supplemented)+1) #сквозной индекс для итоговой таблицы
display(df_supplemented)
df_supplemented.to_excel('HW1_Not_sorted+sorted.xlsx')
#После пересортировки найдено 525 видео (незначительная разница с 519), попробуем сбор видео по временным периодам

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,5DC3p7KmZB248TDo9ceNNKVPPTU,youtube#video,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,где купить шоппер: у меня в директе инстаграма...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,https://i.ytimg.com/vi/phGsMQHOGD8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/phGsMQHOGD8/hqdefault.jpg,480,360,olya cass,none,2019-08-13T14:00:08Z
2,youtube#searchResult,Dcc1plzAH0gPDFNndUK8GIhdByc,youtube#video,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,https://i.ytimg.com/vi/HmgvJrf9wFI/mqdefault.jpg,320,180,https://i.ytimg.com/vi/HmgvJrf9wFI/hqdefault.jpg,480,360,momocka,none,2016-06-05T10:09:33Z
3,youtube#searchResult,068hYbbg5jUT13zL_0eVFnBRjtQ,youtube#video,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,https://i.ytimg.com/vi/iaaCHWUPAk8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/iaaCHWUPAk8/hqdefault.jpg,480,360,Эллина Дейли / Ellina Daily,none,2016-04-26T13:26:18Z
4,youtube#searchResult,bC1jMdVjZ94iIxTDXIkJXsgFu-Y,youtube#video,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,Как сортировать мусор? Где искать в России инф...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,https://i.ytimg.com/vi/nx63IlOrqqE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/nx63IlOrqqE/hqdefault.jpg,480,360,ulielie,none,2019-05-01T13:00:17Z
5,youtube#searchResult,qE9JwvA5uaONYXTiHwHB2nig9KU,youtube#video,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,https://i.ytimg.com/vi/9oPYEDwpDTs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9oPYEDwpDTs/hqdefault.jpg,480,360,LANA,none,2019-09-12T07:37:35Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,youtube#searchResult,IuaaEILZv6fLwO5AYLwB5z3SgbI,youtube#video,dLbOK_ecQq4,2021-02-13T06:18:27Z,UCzHHETVL7NB01VMn45HzIhw,Комплекс упражнений от головной боли,"Студия Aktiv.pro создана для тех, кто любит дв...",https://i.ytimg.com/vi/dLbOK_ecQq4/default.jpg,120,90,https://i.ytimg.com/vi/dLbOK_ecQq4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/dLbOK_ecQq4/hqdefault.jpg,480,360,Студия Aktiv.Pro Надежды Соколовой,none,2021-02-13T06:18:27Z
522,youtube#searchResult,117w2EGqJA7WcrjB-j-KkNKp9vg,youtube#video,2vSWwDOlhRE,2021-02-06T15:50:23Z,UCu7j7rFcFYkFSZWc7T7uFgA,Как я *НА САМОМ ДЕЛЕ* поддерживаю ПОРЯДОК с ма...,Я уже достаточно давно трачу на уборку 20 мину...,https://i.ytimg.com/vi/2vSWwDOlhRE/default.jpg,120,90,https://i.ytimg.com/vi/2vSWwDOlhRE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/2vSWwDOlhRE/hqdefault.jpg,480,360,Julia Bandak,none,2021-02-06T15:50:23Z
523,youtube#searchResult,MjzKExh3EW7lZBdsBljLnFJqLcc,youtube#video,IFveqNK8-gM,2021-02-11T10:22:02Z,UC-IOGBUzqF8nSiS9L7fMjxA,"SUB)미니멀라이프, 쉬운 청소방법&amp;우리집 청소도구/Minimalist Cl...","자막이 겹쳐보이거나, 불필요하시면 영상 설정에서 '자막사용안함(끔)'을 클릭해주세요...",https://i.ytimg.com/vi/IFveqNK8-gM/default.jpg,120,90,https://i.ytimg.com/vi/IFveqNK8-gM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/IFveqNK8-gM/hqdefault.jpg,480,360,밀리카Milika,none,2021-02-11T10:22:02Z
524,youtube#searchResult,GF8RCh2Q67bEQdg2

In [41]:
df_supplemented = pandas.read_excel('HW1_Not_sorted+sorted.xlsx', index_col=0)
#возврат к выдаче без учета date

In [43]:
#Запросы без включения аргумента order, но с учетом даты
#Первый заход
request = youtube.search().list(
part="snippet",
maxResults=50,
q=q,
type=type,
publishedAfter=publishedAfter,
publishedBefore=publishedBefore
)
response = request.execute()
#Занесение собранных данных в итоговую таблицу df_supplemented
df_supplemented = pandas.json_normalize(response['items'])
#Цикл для прохода по всем следующим таблицам с выдачей
i=0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
    part="snippet",
    maxResults=50,
    q=q,
    type=type,
    publishedAfter=publishedAfter,
    publishedBefore=publishedBefore,
    pageToken = response['nextPageToken']
    )
    response = request.execute()
    #Для визуализации процесса
    print(f'Итерация №{i}, "date"{publishedAfter[:11]}-{publishedBefore[:11]}')
    i+=1
    df_additional = pandas.json_normalize(response['items'])
    df_supplemented = pandas.concat([df_supplemented, df_additional])
    #Занесение собранных данных в итоговую таблицу df_supplemented
    df_additional = pandas.json_normalize(response['items'])
    df_supplemented = pandas.concat([df_supplemented, df_additional])
print(f"Искомых объектов {response['pageInfo']['totalResults']}, \
       а найденных БЕЗ включения каких-либо значений аргемнта order {len(df_supplemented.drop_duplicates('id.videoId'))}")
#Запрос без включения аргумента order, но с учетом даты c 2000 года по текущее время нашел 521 видео (незначительная разница с 519)
#Предполагаю, что цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи, не даст большого числа видео
#Рассмотрим выдачу по годам, начиная с самого раннего года публикации найденного видео

Итерация №0, "date"2000-01-01T-2021-04-01T
Итерация №1, "date"2000-01-01T-2021-04-01T
Итерация №2, "date"2000-01-01T-2021-04-01T
Итерация №3, "date"2000-01-01T-2021-04-01T
Итерация №4, "date"2000-01-01T-2021-04-01T
Итерация №5, "date"2000-01-01T-2021-04-01T
Итерация №6, "date"2000-01-01T-2021-04-01T
Итерация №7, "date"2000-01-01T-2021-04-01T
Итерация №8, "date"2000-01-01T-2021-04-01T
Итерация №9, "date"2000-01-01T-2021-04-01T
Итерация №10, "date"2000-01-01T-2021-04-01T
Искомых объектов 56162,        а найденных БЕЗ включения каких-либо значений аргемнта order 521


In [50]:
#Проверка совпадения двух столбцов из выдачи
df_supplemented = pandas.read_excel('HW1_Not_sorted+sorted.xlsx', index_col=0)
#возврат к выдаче без учета date
display(
df_supplemented,
df_supplemented['snippet.publishedAt']==df_supplemented['snippet.publishTime'],
f"Совпадение двух столбцов:{sum(df_supplemented['snippet.publishedAt']==df_supplemented['snippet.publishTime'])}")

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,5DC3p7KmZB248TDo9ceNNKVPPTU,youtube#video,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,где купить шоппер: у меня в директе инстаграма...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,https://i.ytimg.com/vi/phGsMQHOGD8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/phGsMQHOGD8/hqdefault.jpg,480,360,olya cass,none,2019-08-13T14:00:08Z
2,youtube#searchResult,Dcc1plzAH0gPDFNndUK8GIhdByc,youtube#video,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,https://i.ytimg.com/vi/HmgvJrf9wFI/mqdefault.jpg,320,180,https://i.ytimg.com/vi/HmgvJrf9wFI/hqdefault.jpg,480,360,momocka,none,2016-06-05T10:09:33Z
3,youtube#searchResult,068hYbbg5jUT13zL_0eVFnBRjtQ,youtube#video,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,https://i.ytimg.com/vi/iaaCHWUPAk8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/iaaCHWUPAk8/hqdefault.jpg,480,360,Эллина Дейли / Ellina Daily,none,2016-04-26T13:26:18Z
4,youtube#searchResult,bC1jMdVjZ94iIxTDXIkJXsgFu-Y,youtube#video,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,Как сортировать мусор? Где искать в России инф...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,https://i.ytimg.com/vi/nx63IlOrqqE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/nx63IlOrqqE/hqdefault.jpg,480,360,ulielie,none,2019-05-01T13:00:17Z
5,youtube#searchResult,qE9JwvA5uaONYXTiHwHB2nig9KU,youtube#video,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,https://i.ytimg.com/vi/9oPYEDwpDTs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9oPYEDwpDTs/hqdefault.jpg,480,360,LANA,none,2019-09-12T07:37:35Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,youtube#searchResult,IuaaEILZv6fLwO5AYLwB5z3SgbI,youtube#video,dLbOK_ecQq4,2021-02-13T06:18:27Z,UCzHHETVL7NB01VMn45HzIhw,Комплекс упражнений от головной боли,"Студия Aktiv.pro создана для тех, кто любит дв...",https://i.ytimg.com/vi/dLbOK_ecQq4/default.jpg,120,90,https://i.ytimg.com/vi/dLbOK_ecQq4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/dLbOK_ecQq4/hqdefault.jpg,480,360,Студия Aktiv.Pro Надежды Соколовой,none,2021-02-13T06:18:27Z
522,youtube#searchResult,117w2EGqJA7WcrjB-j-KkNKp9vg,youtube#video,2vSWwDOlhRE,2021-02-06T15:50:23Z,UCu7j7rFcFYkFSZWc7T7uFgA,Как я *НА САМОМ ДЕЛЕ* поддерживаю ПОРЯДОК с ма...,Я уже достаточно давно трачу на уборку 20 мину...,https://i.ytimg.com/vi/2vSWwDOlhRE/default.jpg,120,90,https://i.ytimg.com/vi/2vSWwDOlhRE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/2vSWwDOlhRE/hqdefault.jpg,480,360,Julia Bandak,none,2021-02-06T15:50:23Z
523,youtube#searchResult,MjzKExh3EW7lZBdsBljLnFJqLcc,youtube#video,IFveqNK8-gM,2021-02-11T10:22:02Z,UC-IOGBUzqF8nSiS9L7fMjxA,"SUB)미니멀라이프, 쉬운 청소방법&amp;우리집 청소도구/Minimalist Cl...","자막이 겹쳐보이거나, 불필요하시면 영상 설정에서 '자막사용안함(끔)'을 클릭해주세요...",https://i.ytimg.com/vi/IFveqNK8-gM/default.jpg,120,90,https://i.ytimg.com/vi/IFveqNK8-gM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/IFveqNK8-gM/hqdefault.jpg,480,360,밀리카Milika,none,2021-02-11T10:22:02Z
524,youtube#searchResult,GF8RCh2Q67bEQdg2

1      True
2      True
3      True
4      True
5      True
       ... 
521    True
522    True
523    True
524    True
525    True
Length: 525, dtype: bool

'Совпадение двух столбцов:525'

In [51]:
#Lambda-функция для перезаписи в столбец snippet.publishedAt только года
df_supplemented['snippet.publishedAt']==df_supplemented['snippet.publishedAt'].apply(lambda yyyy: int(yyyy[:4]))
df_supplemented['snippet.publishedAt']

1      2019-08-13T14:00:08Z
2      2016-06-05T10:09:33Z
3      2016-04-26T13:26:18Z
4      2019-05-01T13:00:17Z
5      2019-09-12T07:37:35Z
               ...         
521    2021-02-13T06:18:27Z
522    2021-02-06T15:50:23Z
523    2021-02-11T10:22:02Z
524    2021-02-09T00:57:15Z
525    2021-02-08T21:00:05Z
Name: snippet.publishedAt, Length: 525, dtype: object

In [52]:
#Поиск самого раннего года
min(df_supplemented['snippet.publishedAt'])

'2010-10-30T12:16:41Z'

In [57]:
#Запросы без включения аргумента order, но с учетом даты
i=0
#year = 2010
# year=year после исчерпания quotas ключа
while year < 2023:
    #Первый заход с включением аргумента date
    request = youtube.search().list(
    part="snippet",
    maxResults=50,
    q=q,
    type=type,
    publishedAfter=f'{year}-01-01T00:00:00Z'
    #publishedBefore=publishedBefore
    )
    response = request.execute()
    #Занесение собранных данных в итоговую таблицу df_supplemented
    df_additional = pandas.json_normalize(response['items'])
    df_supplemented = pandas.concat([df_supplemented, df_additional])
    #Цикл для прохода по всем следующим таблицам с выдачей
    i=0
    while 'nextPageToken' in response.keys():
        request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q=q,
        type=type,
        publishedAfter=f'{year}-01-01T00:00:00Z',
        #publishedBefore=publishedBefore,
        pageToken = response['nextPageToken']
        )
        response = request.execute()
        #Для визуализации процесса
        print(f'Итерация №{i}, "date"{year}')
        i+=1
        df_additional = pandas.json_normalize(response['items'])
        df_supplemented = pandas.concat([df_supplemented, df_additional])
        #Занесение собранных данных в итоговую таблицу df_supplemented
        df_additional = pandas.json_normalize(response['items'])
        df_supplemented = pandas.concat([df_supplemented, df_additional])
    print(f"Искомых объектов с {year} года {response['pageInfo']['totalResults']}, \
       а найденных БЕЗ включения каких-либо значений аргемнта order {len(df_supplemented.drop_duplicates('id.videoId'))}")
    year+=1
#Начиная с 2010 года было найдено 1232 видео, что удовлетворяет условиям задания

Итерация №0, "date"2022
Итерация №1, "date"2022
Итерация №2, "date"2022
Итерация №3, "date"2022
Итерация №4, "date"2022
Итерация №5, "date"2022
Итерация №6, "date"2022
Итерация №7, "date"2022
Итерация №8, "date"2022
Итерация №9, "date"2022
Итерация №10, "date"2022
Искомых объектов с 2022 года 55917,        а найденных БЕЗ включения каких-либо значений аргемнта order 1232


In [58]:
#Удалим дубликаты видео по их ID и запишем в Excel
df_supplemented = df_supplemented.drop_duplicates('id.videoId')
df_supplemented.index = range(1, len(df_supplemented)+1) #сквозной индекс для итоговой таблицы
display(df_supplemented)
df_supplemented.to_excel('HW1_Not_sorted+sorted+date.xlsx')

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,5DC3p7KmZB248TDo9ceNNKVPPTU,youtube#video,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,где купить шоппер: у меня в директе инстаграма...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,https://i.ytimg.com/vi/phGsMQHOGD8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/phGsMQHOGD8/hqdefault.jpg,480,360,olya cass,none,2019-08-13T14:00:08Z
2,youtube#searchResult,Dcc1plzAH0gPDFNndUK8GIhdByc,youtube#video,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,https://i.ytimg.com/vi/HmgvJrf9wFI/mqdefault.jpg,320,180,https://i.ytimg.com/vi/HmgvJrf9wFI/hqdefault.jpg,480,360,momocka,none,2016-06-05T10:09:33Z
3,youtube#searchResult,068hYbbg5jUT13zL_0eVFnBRjtQ,youtube#video,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,https://i.ytimg.com/vi/iaaCHWUPAk8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/iaaCHWUPAk8/hqdefault.jpg,480,360,Эллина Дейли / Ellina Daily,none,2016-04-26T13:26:18Z
4,youtube#searchResult,bC1jMdVjZ94iIxTDXIkJXsgFu-Y,youtube#video,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,Как сортировать мусор? Где искать в России инф...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,https://i.ytimg.com/vi/nx63IlOrqqE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/nx63IlOrqqE/hqdefault.jpg,480,360,ulielie,none,2019-05-01T13:00:17Z
5,youtube#searchResult,qE9JwvA5uaONYXTiHwHB2nig9KU,youtube#video,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,https://i.ytimg.com/vi/9oPYEDwpDTs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9oPYEDwpDTs/hqdefault.jpg,480,360,LANA,none,2019-09-12T07:37:35Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,youtube#searchResult,XQag9_1JBdEcU0OMC5M_bAOX_mc,youtube#video,d2vWpB4_7cA,2018-11-26T09:16:39Z,UCuqDp0mvGgTTQEBGtvUz8Sg,Эко-жизнь!!!Эко продукция 21века!Все вопросы н...,,https://i.ytimg.com/vi/d2vWpB4_7cA/default.jpg,120,90,https://i.ytimg.com/vi/d2vWpB4_7cA/mqdefault.jpg,320,180,https://i.ytimg.com/vi/d2vWpB4_7cA/hqdefault.jpg,480,360,Екатерина GREENWAY,none,2018-11-26T09:16:39Z
1229,youtube#searchResult,j9cHWXio3Ew8nxewQcPE4DTmSR8,youtube#video,aaHSoQluQ5U,2019-12-02T07:35:21Z,UCChRlWpFid2TcHJ8evIZedg,Осознанное потребление,,https://i.ytimg.com/vi/aaHSoQluQ5U/default.jpg,120,90,https://i.ytimg.com/vi/aaHSoQluQ5U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/aaHSoQluQ5U/hqdefault.jpg,480,360,mashulya love93,none,2019-12-02T07:35:21Z
1230,youtube#searchResult,szMYhphUoE6KI38LbfK5lN1rGnM,youtube#video,n2E4fVFP1HQ,2015-10-16T16:02:49Z,UCJ04uWyEYcA3YqTPqwGXq0Q,ЭКО-Содействие октябрь 2015,1 минута ЭКО-Содействие.,https://i.ytimg.com/vi/n2E4fVFP1HQ/default.jpg,120,90,https://i.ytimg.com/vi/n2E4fVFP1HQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/n2E4fVFP1HQ/hqdefault.jpg,480,360,"Эко-содействие, Центр Планирования Семьи",none,2015-10-16T16:02:49Z
1231,youtube#searchResult,1hNFml9ST6Tq-5WW0CGGC_v27nU,youtube#video,X89jqM0V9lk,2018-03-28T08:16:50Z,UCbjbNOB-oRV77CLS8SrjbFQ,Простая утренняя привычка поможет найти решени...,Простая

In [3]:
#Считываем файл с видео Excel и создаем список с ID видео
videoId_df = pandas.read_excel('HW1_Not_sorted+sorted+date.xlsx', index_col=0)
display(videoId_df)
videoId_list = videoId_df[f'id.videoId'].to_list()
videoId_list

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,5DC3p7KmZB248TDo9ceNNKVPPTU,youtube#video,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,где купить шоппер: у меня в директе инстаграма...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,https://i.ytimg.com/vi/phGsMQHOGD8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/phGsMQHOGD8/hqdefault.jpg,480,360,olya cass,none,2019-08-13T14:00:08Z
2,youtube#searchResult,Dcc1plzAH0gPDFNndUK8GIhdByc,youtube#video,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,https://i.ytimg.com/vi/HmgvJrf9wFI/mqdefault.jpg,320,180,https://i.ytimg.com/vi/HmgvJrf9wFI/hqdefault.jpg,480,360,momocka,none,2016-06-05T10:09:33Z
3,youtube#searchResult,068hYbbg5jUT13zL_0eVFnBRjtQ,youtube#video,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,https://i.ytimg.com/vi/iaaCHWUPAk8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/iaaCHWUPAk8/hqdefault.jpg,480,360,Эллина Дейли / Ellina Daily,none,2016-04-26T13:26:18Z
4,youtube#searchResult,bC1jMdVjZ94iIxTDXIkJXsgFu-Y,youtube#video,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,Как сортировать мусор? Где искать в России инф...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,https://i.ytimg.com/vi/nx63IlOrqqE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/nx63IlOrqqE/hqdefault.jpg,480,360,ulielie,none,2019-05-01T13:00:17Z
5,youtube#searchResult,qE9JwvA5uaONYXTiHwHB2nig9KU,youtube#video,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,https://i.ytimg.com/vi/9oPYEDwpDTs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9oPYEDwpDTs/hqdefault.jpg,480,360,LANA,none,2019-09-12T07:37:35Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,youtube#searchResult,XQag9_1JBdEcU0OMC5M_bAOX_mc,youtube#video,d2vWpB4_7cA,2018-11-26T09:16:39Z,UCuqDp0mvGgTTQEBGtvUz8Sg,Эко-жизнь!!!Эко продукция 21века!Все вопросы н...,NaN,https://i.ytimg.com/vi/d2vWpB4_7cA/default.jpg,120,90,https://i.ytimg.com/vi/d2vWpB4_7cA/mqdefault.jpg,320,180,https://i.ytimg.com/vi/d2vWpB4_7cA/hqdefault.jpg,480,360,Екатерина GREENWAY,none,2018-11-26T09:16:39Z
1229,youtube#searchResult,j9cHWXio3Ew8nxewQcPE4DTmSR8,youtube#video,aaHSoQluQ5U,2019-12-02T07:35:21Z,UCChRlWpFid2TcHJ8evIZedg,Осознанное потребление,NaN,https://i.ytimg.com/vi/aaHSoQluQ5U/default.jpg,120,90,https://i.ytimg.com/vi/aaHSoQluQ5U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/aaHSoQluQ5U/hqdefault.jpg,480,360,mashulya love93,none,2019-12-02T07:35:21Z
1230,youtube#searchResult,szMYhphUoE6KI38LbfK5lN1rGnM,youtube#video,n2E4fVFP1HQ,2015-10-16T16:02:49Z,UCJ04uWyEYcA3YqTPqwGXq0Q,ЭКО-Содействие октябрь 2015,1 минута ЭКО-Содействие.,https://i.ytimg.com/vi/n2E4fVFP1HQ/default.jpg,120,90,https://i.ytimg.com/vi/n2E4fVFP1HQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/n2E4fVFP1HQ/hqdefault.jpg,480,360,"Эко-содействие, Центр Планирования Семьи",none,2015-10-16T16:02:49Z
1231,youtube#searchResult,1hNFml9ST6Tq-5WW0CGGC_v27nU,youtube#video,X89jqM0V9lk,2018-03-28T08:16:50Z,UCbjbNOB-oRV77CLS8SrjbFQ,Простая утренняя привычка поможет найти решени...,П

['phGsMQHOGD8',
 'HmgvJrf9wFI',
 'iaaCHWUPAk8',
 'nx63IlOrqqE',
 '9oPYEDwpDTs',
 '2n9ZilH3EA0',
 'l1I45bWh9pg',
 'f0w3M4LzVBU',
 'VihnsI_7jJM',
 'raBiKFvbCpY',
 'C3TAr_Blc5U',
 'JxRw5d80A3Y',
 'zbxA6PmEfCU',
 'SHLywXroShc',
 '71ctY3KAnD8',
 'dLNWIkgk_Lc',
 'jAgOSar20iA',
 'JopVOrz7PWs',
 '9Y2rrA9eXbg',
 'jN9SfilgSMM',
 '60U-2k77GKY',
 'aOuGctVNwr4',
 'dDangN85L_Y',
 'Iu81BR44HhE',
 'n95LvAPV6LI',
 'NUe64uJG-1Q',
 'VK52mHhHuXU',
 'ihT4rOtJgpk',
 'fFBVN4x4kw0',
 'gJAp5tWiN8Q',
 'Eaahsd3zaCQ',
 '_xmkIPW4Wg8',
 'mfk5GH-FPsI',
 'S91hjdmGQnk',
 'zFr3TNQ4Ung',
 'L40WAi3ut28',
 'dVNSkoVVR6w',
 '6dQucwUXx3E',
 'ezFvlLgDH6o',
 '-cKwKte2jt0',
 'XwJMrXyXWcA',
 'nUfp5mCOf1Q',
 'r4oo4d3kSnE',
 'z-7hDohQBIc',
 'PSfRmB5sVQs',
 'ODcfsSy6H_A',
 'LqrLHOnEyvw',
 '3WlpqGOyXLE',
 'RzQzm7eSbek',
 'MHDrjV4ua1E',
 'j7tGd68gEC8',
 '0BHzYDs8n1o',
 'QDihULkpRZ4',
 'S1cHoW640Gs',
 'AIFxnlvOzPw',
 'FxWwi1Z-mhI',
 'p-mzytwPylg',
 'Artbpr02tMw',
 'EfXfEH7lB2Y',
 'nWEjp0QI6ds',
 'XPj2wrkRSeU',
 'zyUk-zAQGHo',
 'hNaZp9

In [4]:
#Подаем ID первых 50 видео из выдачи для использования метода videos
request = youtube.videos().list(
    part=["snippet",
    "contentDetails",
    "id",
    "localizations",
    "statistics",
    "status",
    "topicDetails"],
    id=videoId_list[0:50],
    maxResults=50
)
response = request.execute()
df_supplemented = pandas.json_normalize(response['items'])
df_supplemented

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,snippet.thumbnails.maxres.width,snippet.thumbnails.maxres.height,snippet.defaultAudioLanguage,snippet.defaultLanguage,localizations.ru.title,localizations.ru.description,localizations.uk.title,localizations.uk.description,localizations.en.title,localizations.en.description
0,youtube#video,affCrRR1L22jYdRhMkKyMuNh49A,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,мой инстаграм - olya.cass\nмой секонд - https:...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,6n4-820uSMHi3wkoIIgFnueJmDI,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,...,1280.0,720.0,ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#video,FpEa9EGZKP3Nn7R6zYdtRqM0xCI,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,...,NaN,NaN,en,ru,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",NaN,NaN,NaN,NaN
3,youtube#video,P_3gu1bRIxZTWhO5ovvsHt4B8tY,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,🌿 Как сортировать мусор? Где искать в России и...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,...,1280.0,720.0,ru,ru,♻️ Полезные ЭКО привычки: простая сортировка м...,🌿 Как сортировать мусор? Где искать в России и...,NaN,NaN,NaN,NaN
4,youtube#video,KSwjR2i-oZaIldUADLqZ9WeMdOQ,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,...,1280.0,720.0,ru,ru,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",NaN,NaN,NaN,NaN
5,youtube#video,mF4McZsQDr9EfxQntT9jm8GXhfA,2n9ZilH3EA0,2019-12-15T07:57:48Z,UCvaRU1187uKgq2I0XdxDxdw,"7 немодных эко-привычек | пересмотр ""популярно...","Пока Грету Тунберг признали персоной года, а б...",https://i.ytimg.com/vi/2n9ZilH3EA0/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,youtube#video,8VLsZVN80GMfFR535tznDOnynWg,l1I45bWh9pg,2020-01-20T18:15:00Z,UCAVJJsN5ZGldRt6xkGxdzlA,ЭКО-ПРИВЫЧКИ ДЛЯ НАЧИНАЮЩИХ | zero waste | Areen🌏,"☀\nЯ очень надеюсь, что тебе понравилось это в...",https://i.ytimg.com/vi/l1I45bWh9pg/default.jpg,120,90,...,1280.0,720.0,ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,youtube#video,q07mKNA4Qz_s1YmaqWLSHi6k8nY,f0w3M4LzVBU,2020-07-01T17:55:48Z,UCF3qPVM0AO--3ZHIyo5aIEQ,Экологичные привычки / ЭкоХакеры #1,"Представь, что ты решился изменить свою жизни ...",https://i.ytimg.com/vi/f0w3M4LzVBU/default.jpg,120,90,...,1280.0,720.0,NaN,ru,Экологичные привычки / ЭкоХакеры #1,"Представь, что ты решился изменить свою жизни ...",NaN,NaN,NaN,NaN
8,youtube#video,IIP_jNeJC2mdyDNBrMJFMl6un90,VihnsI_7jJM,2019-10-16T10:31:01Z,UCEHBNDyOWEjzzeg6lQWdV_A,МОИ ЭКОПРИВЫЧКИ НА КУХНЕ И ДОМА • ZERO WASTE,"Продолжаю говорить на тему экологии, zero wast...",https://i.ytimg.com/vi/VihnsI_7jJM/default.jpg,120,90,...,1280.0,720.0,ru,ru,МОИ ЭКОПРИВЫЧКИ НА КУХНЕ И ДОМА • ZERO WASTE,"Продолжаю говорить на тему экологии, zero wast...",NaN,NaN,NaN,NaN
9,youtube#video,UGyYmLYn2MYiMkzgNND7PEihFu4,raBiKFvbCpY,2019-08-24T05:56:44Z,UCw2kqV23_ZMLn5jIILLAc1A,Zero & Low Waste лайфхаки/ Полезные альтернати...,Сегодня вещаю о важной теме для сокращения кол...,https://i.ytimg.com/vi/raBiKFvbCpY/default.jpg,120,90,...,1280.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Подаем ID всех видео из выдачи для использования метода videos
i = 0
for i in range (50, len(videoId_list)+1, 50):
    request = youtube.videos().list(
    part=["snippet",
    "contentDetails",
    "id",
    "localizations",
    "statistics",
    "status",
    "topicDetails"],
    id=videoId_list[i:i+50],
    maxResults=50
    )
    response = request.execute()
    print(f'Итерация №{i}')
    df_additional = pandas.json_normalize(response['items'])
    df_supplemented = pandas.concat([df_supplemented, df_additional])
df_supplemented

Итерация №50
Итерация №100
Итерация №150
Итерация №200
Итерация №250
Итерация №300
Итерация №350
Итерация №400
Итерация №450
Итерация №500
Итерация №550
Итерация №600
Итерация №650
Итерация №700
Итерация №750
Итерация №800
Итерация №850
Итерация №900
Итерация №950
Итерация №1000
Итерация №1050
Итерация №1100
Итерация №1150
Итерация №1200


,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,localizations.nl.title,localizations.nl.description,localizations.sv.title,localizations.sv.description,localizations.el.title,localizations.el.description,localizations.fi.title,localizations.fi.description,localizations.kk.title,localizations.kk.description
0,youtube#video,affCrRR1L22jYdRhMkKyMuNh49A,phGsMQHOGD8,2019-08-13T14:00:08Z,UCOh0R9lzX9Jz6zA5p3d1rZA,16 моих эко-привычек,мой инстаграм - olya.cass\nмой секонд - https:...,https://i.ytimg.com/vi/phGsMQHOGD8/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,6n4-820uSMHi3wkoIIgFnueJmDI,HmgvJrf9wFI,2016-06-05T10:09:33Z,UCDMIkGZILWrY7rDhwAnfJWg,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",https://i.ytimg.com/vi/HmgvJrf9wFI/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#video,FpEa9EGZKP3Nn7R6zYdtRqM0xCI,iaaCHWUPAk8,2016-04-26T13:26:18Z,UCZrC50ibKjBm84ZDEr6Yn7Q,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",https://i.ytimg.com/vi/iaaCHWUPAk8/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#video,P_3gu1bRIxZTWhO5ovvsHt4B8tY,nx63IlOrqqE,2019-05-01T13:00:17Z,UCC7E0reSUtQEolCi1rgBG1Q,♻️ Полезные ЭКО привычки: простая сортировка м...,🌿 Как сортировать мусор? Где искать в России и...,https://i.ytimg.com/vi/nx63IlOrqqE/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#video,KSwjR2i-oZaIldUADLqZ9WeMdOQ,9oPYEDwpDTs,2019-09-12T07:37:35Z,UCOd3gr63GIL9Kwb_kLSrZvg,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",https://i.ytimg.com/vi/9oPYEDwpDTs/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,youtube#video,sFV-x1i0JlBhztwfYVhw-0MSm9M,d2vWpB4_7cA,2018-11-26T09:16:39Z,UCuqDp0mvGgTTQEBGtvUz8Sg,Эко-жизнь!!!Эко продукция 21века!Все вопросы н...,,https://i.ytimg.com/vi/d2vWpB4_7cA/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,youtube#video,yDfoMNVh6JSRIYr3H6al1qBDbwQ,aaHSoQluQ5U,2019-12-02T07:35:21Z,UCChRlWpFid2TcHJ8evIZedg,Осознанное потребление,,https://i.ytimg.com/vi/aaHSoQluQ5U/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,youtube#video,P9J2dbPeVvnlv2buFRI_Aq1B1O4,n2E4fVFP1HQ,2015-10-16T16:02:49Z,UCJ04uWyEYcA3YqTPqwGXq0Q,ЭКО-Содействие октябрь 2015,1 минута ЭКО-Содействие,https://i.ytimg.com/vi/n2E4fVFP1HQ/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,youtube#video,5EOFYlDpGBKXVEgNfaGkWhNnowQ,X89jqM0V9lk,2018-03-28T08:16:50Z,UCbjbNOB-oRV77CLS8SrjbFQ,Простая утренняя привычка поможет найти решени...,Простая утренняя привычка поможет найти решени...,https://i.ytimg.com/vi/X89jqM0V9lk/default.jpg,120,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Удалим дубликаты видео по их ID  и нужные столбцы и запишем в Excel
df_supplemented = df_supplemented[['id', 'snippet.title', 'snippet.description', 'snippet.publishedAt', 'contentDetails.duration',
'snippet.channelTitle', 'statistics.viewCount', 'statistics.likeCount', 'statistics.dislikeCount',
'statistics.commentCount']]
df_supplemented = df_supplemented.drop_duplicates('id')
df_supplemented.index = range(1, len(df_supplemented)+1) #сквозной индекс для итоговой таблицы
display(df_supplemented)
df_supplemented.to_excel('HW1_Video.xlsx')

,id,snippet.title,snippet.description,snippet.publishedAt,contentDetails.duration,snippet.channelTitle,statistics.viewCount,statistics.likeCount,statistics.dislikeCount,statistics.commentCount
1,phGsMQHOGD8,16 моих эко-привычек,мой инстаграм - olya.cass\nмой секонд - https:...,2019-08-13T14:00:08Z,PT21M25S,olya cass,144540,8705,377,501
2,HmgvJrf9wFI,Как Спасти Нашу Планету? | Эко-Привычки,"Прежде, чем задать вопрос, пожалуйста, ПРОЧИТА...",2016-06-05T10:09:33Z,PT12M41S,momocka,90063,6796,372,699
3,iaaCHWUPAk8,"Экология, полезные привычки, zero waste, миним...","О том, как я веду дружелюбный по отношении к п...",2016-04-26T13:26:18Z,PT9M29S,Эллина Дейли / Ellina Daily,84862,3847,61,249
4,nx63IlOrqqE,♻️ Полезные ЭКО привычки: простая сортировка м...,🌿 Как сортировать мусор? Где искать в России и...,2019-05-01T13:00:17Z,PT13M56S,ulielie,66136,5127,46,737
5,9oPYEDwpDTs,ВОЗМОЖЕН ЛИ ZERO WASTE? - Эко-привычки и осозн...,"Все слышали о zero waste или ноль отходов, о с...",2019-09-12T07:37:35Z,PT4M31S,LANA,61537,5748,54,326
...,...,...,...,...,...,...,...,...,...,...
1222,d2vWpB4_7cA,Эко-жизнь!!!Эко продукция 21века!Все вопросы н...,,2018-11-26T09:16:39Z,PT2M23S,Екатерина GREENWAY,2,0,0,0
1223,aaHSoQluQ5U,Осознанное потребление,,2019-12-02T07:35:21Z,PT50S,mashulya love93,701,28,1,4
1224,n2E4fVFP1HQ,ЭКО-Содействие октябрь 2015,1 минута ЭКО-Содействие,2015-10-16T16:02:49Z,PT1M3S,"Эко-содействие, Центр Планирования Семьи",104,1,0,0
1225,X89jqM0V9lk,Простая утренняя привычка поможет найти решени...,Простая утренняя привычка поможет найти решени...,2018-03-28T08:16:50Z,PT1M29S,Постановка и достижение целей Видео уроки Иван...,467,35,1,11
